### Preproc EMU....

In [1]:
# Edit here:
subj = {'id' :'128', 'emu': 'EMU128'}
edf_path = '/Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EMU128/EMU128_FULL.EDF'


In [2]:
import mne, os, ast, logging
import pandas as pd
import numpy as np
from os.path import join
from autoreject import get_rejection_threshold
from matplotlib import pyplot as plt
from datetime import datetime

from expt_params import *
from functions import *



In [3]:
channels_info_path = join(params['channels_info_dir'], f'out_EMU{subj['id']}.xlsx')

Logging Stuff (later)

In [13]:

mne.set_log_level('warning')

# ------------------- subject-specific data paths --------------
channels_info_path = join(params['channels_info_dir'], f'out_Subject_{subj['id']}.xlsx')
logfile_path = join(params['logfiles_dir'], f'{subj['id']}_Trial_RTs.csv')

# ------------------ output dirs and logging
outdir_logging = join(params['outdir_logging'], subj['emu'])
if not os.path.isdir(outdir_logging):
    os.mkdir(outdir_logging)

logging.basicConfig(
    filename= join(outdir_logging, f'{subj['emu']}.log'),
    filemode='a',  # Append mode to keep all logs
    format='%(message)s',
    level=logging.INFO)

# save experiment params to log
now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
logging.info(now)


for key, val in params.items():
    logging.info(f"{key}: {val}")



In [4]:
# ------------------- Load data ---------------------------- 
print('reading edf...')
edf = mne.io.read_raw_edf(edf_path, preload=False)
print('Done')


edf.crop(tmin=0, tmax=3600)

edf.load_data()


reading edf...
Extracting EDF parameters from /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EMU128/EMU128_FULL.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/var/folders/t3/xx1drn4x5p74xhh3vv6lwqsr0000gn/T/ipykernel_12423/1966584490.py:3: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  edf = mne.io.read_raw_edf(edf_path, preload=False)


Done
Reading 0 ... 7372800  =      0.000 ...  3600.000 secs...


<RawEDF | EMU128_FULL.EDF, 148 x 7372801 (3600.0 s), ~8.13 GB, data loaded>

In [5]:
# if resample is specified in the expt_params:
if params['resample_fs']:
    print('resampling...')
    edf.resample(params['resample_fs'])
    print('Done')


resampling...
Done


In [7]:

# get channel names from pandas df
df_channel_info = pd.read_excel(channels_info_path)
channel_names = np.array(df_channel_info.Channel)

print('N channels =', len(channel_names))
#logging.info(f"N channels: {len(channel_names)}")


N channels = 119


In [8]:
# pick the channels based on anatomy csv
edf.pick_channels(channel_names)
assert len(edf.info['ch_names']) == len(channel_names)

# correct channel types
edf.set_channel_types({ch_n:'seeg' for ch_n in edf.ch_names})


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | EMU128_FULL.EDF, 119 x 3600000 (3600.0 s), ~3.19 GB, data loaded>

In [9]:
# ----------------- Add montage ------------------------------
montage = get_montage_from_bs(electrode_data=df_channel_info, 
                              electrode_space=params['electrode_space'], 
                              subjects_dir=params['subjects_dir'])
edf.set_montage(montage)
#_fig = mne.viz.plot_montage(montage, show=False)
#_fig.suptitle(subj['emu'])
#_fig.savefig(join(outdir_logging, f'{subj['emu']}_montage.jpg'))


<RawEDF | EMU128_FULL.EDF, 119 x 3600000 (3600.0 s), ~3.19 GB, data loaded>

In [10]:
# ------------------- Filters -------------------------------
print('Applying filters...')
edf = apply_standard_filters(edf, params['high_pass_cutoff'], params['notch_freqs'])
print('Done')

Applying filters...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    5.5s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.8s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    5.5s


Done


### Reject bad channels

In [11]:
# ---------- Interactive plot to reject bads ------------------ #
%matplotlib qt

# --- PSD plot
psd =  edf.compute_psd(fmax=200)
psd.plot()

# --- raw plot
edf.plot()
print('>> Mark bads on raw plot (popup window)')

Effective window size : 2.048 (s)
Plotting power spectral density (dB=True).
Using matplotlib as 2D backend.
>> Mark bads on raw plot (popup window)


Channels marked as bad:
['N12', 'N13', 'N14', 'N15', 'H14', 'H15', 'Y18', 'X18', 'X17']


In [14]:
# --------------------- dropping bad channels here ------------------ #
print('Bad channels:', edf.info['bads'])

# We will append more bad channels later if we find them. 
# We need this list because after we actually drop the channels this will be gone from the info dict
all_bad_channels =  edf.info['bads'] 
edf.drop_channels(edf.info['bads'])


Bad channels: ['N12', 'N13', 'N14', 'N15', 'H14', 'H15', 'P14', 'P15', 'Y18']


<RawEDF | EMU128_FULL.EDF, 110 x 3600000 (3600.0 s), ~2.95 GB, data loaded>

In [13]:
# --------------------- Saving EDF Here ------------------ #
edf.save(join(params['outdir_edfs'], f'{subj['emu']}-3600_eeg.fif'))

Overwriting existing file.
Writing /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EDFs/EMU128-3600_eeg.fif
Closing /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EDFs/EMU128-3600_eeg.fif
[done]


In [ ]:

# ------------------- epochs and baseline correction ------------------ # 
logfile = pd.read_csv(logfile_path)

######
print('WARNING: systematically dropping the last line of logfile')
logfile = logfile[:-1] # drop the last row, I think it's meant to be empty ?
#######

event_times = np.array(logfile.EMU_OnsetTime)

# Convert onset times to sample indices
sfreq = edf.info['sfreq'] 
onset_samples = (event_times * sfreq).astype(int)
event_id = 1 # same for all triggers
events = np.column_stack((onset_samples, np.zeros(len(onset_samples), int), np.full(len(onset_samples), event_id)))

print('First 10 events:\n', events[:10])

epochs = mne.Epochs(edf, 
                    events, 
                    tmin = params['epochs_tmin'],
                    tmax = params['epochs_tmax'],
                    baseline = params['epochs_baseline'],
                    decim = params['epochs_decim'],
                    metadata = logfile,
                    reject = None,
                    preload = True)

logging.info(f'Init num of epochs: {len(epochs)}')

# reject noisy epochs using autoreject:
# see docs here https://autoreject.github.io/stable/explanation.html
rejection_threshold = get_rejection_threshold(epochs, decim=2)
print(rejection_threshold)
logging.info(f'epochs_rejection_threshold: {rejection_threshold}')

epochs_rej = epochs.copy()
epochs_rej.drop_bad(reject=rejection_threshold, verbose='warning')

print_epochs_rejection_info(epochs, epochs_rej)

#--- plot a graph showing how many epochs were rejected due to each channel
# Note: epochs can be rejected based on multiple channels, this only shows the # of times a channel caused a bad epoch
%matplotlib inline
fig = plot_epochs_rejcount_by_channel(epochs_rej, method='zscore', threshold=2)

### Manually determine if there are more bad channels

In [ ]:
# manually iterate over some rejection of channels if needed
drop_channel_then_print_new_epoch_rejcount(epochs=epochs, 
                                           bad_channels=[], 
                                           rejection_threshold=rejection_threshold)

In [ ]:
# Based on the above iterations derermine which channels to reject for the final cleaning
channels_to_drop = [] # if none, leave empty

# append to list of all bad channels which we will add to the logs
for ch in channels_to_drop:
    all_bad_channels.append(ch)

epochs_clean = epochs.copy() # make a copy
epochs_clean.drop_channels(channels_to_drop) # drop the outlier channel(s) based on plot above
epochs_clean.drop_bad(reject=rejection_threshold, verbose='warning') # then reject bad epochs using the threshold
print('WARNING: threshold was calculated using the full set of electrodes, but applied after rejecting some new ones')

num_rej, percent_rej = print_epochs_rejection_info(epochs, epochs_clean)
# plot_epochs_rejcount_by_channel(epochs_clean, method='zscore', threshold=2)


logging.info(f'Bad channels: {all_bad_channels}')
logging.info(f'Num bad epochs: {num_rej} ({round(percent_rej, 2)}%)')
logging.info(f'Num clean epochs: {len(epochs_clean)}')

# save epochs
epochs_clean.save(join(params['outdir_epochs'], f'{subj['emu']}-epo.fif'))



In [ ]:
# plot evoked and savefig
%matplotlib inline
fig_evoked = epochs_clean.average().plot_joint()
fig_evoked.savefig(join(outdir_logging, f'{subj['emu']}_evoked.jpg'))


# finally plot sensors on fsaverage
brain = plot_seeg_freesurfer(epochs_clean, subjects_dir=params['subjects_dir'])
brain.save_image(join(outdir_logging, f'{subj['emu']}_brain.jpg'))
brain.close()


### Hilbert Transform on Raw EDF (Because EMU128 is not task oriented)

In [14]:
frequency_bands = [
    (1, 4, "DELTA"), 
    (4, 8, "THETA"), 
    (8.0, 12.0, "ALPHA"), 
    (12, 30, "BETA"), 
    (30, 50, "LG"), 
    (75, 150, "HG")
]

n_channels = len(edf.ch_names)
n_times = edf.n_times
n_bands = len(frequency_bands)
power_cube = np.zeros((n_channels, n_bands, n_times))

for i, (l, h, name) in enumerate(frequency_bands):
    # Copy to avoid modifying original raw_lowpass
    print("Working on %s now..." % name)
    edf_HT = edf.copy().filter(l_freq=l, h_freq=h, method='iir')
    # Apply Hilbert transform (envelope=True gives the magnitude of the analytic signal)
    edf_HT.apply_hilbert(envelope=True)
    power_cube[:, i, :] = edf_HT.get_data()
    edf_HT.save(join(params['outdir_edfs'], f'{subj['emu']}_{name}-3600_eeg.fif'), overwrite=True)





Working on DELTA now...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 4 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 4.00 Hz: -6.02, -6.02 dB

Overwriting existing file.
Writing /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EDFs/EMU128_DELTA-3600_eeg.fif
Closing /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EDFs/EMU128_DELTA-3600_eeg.fif
[done]
Working on THETA now...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -6.02, -6.02 dB

Overwriting existing file.
Writing /Users/kamronsoldozy/Documents/PhD/ANALYSIS/DATA/EDFs/EMU128_THETA-3600_eeg.fif
Closi

In [15]:
np.save(join(params['outdir_edfs'], f'{subj['emu']}-power_cube.npy'), power_cube)

### Experimental Time 

In [12]:
from seqnmf import seqnmf, plot, example_data
from scipy.ndimage import uniform_filter1d
import time

In [5]:
power_cube = np.load(join(params['outdir_edfs'], f'{subj['emu']}-power_cube.npy'))

In [6]:

### Let's do a sliding z-score on this data for each channel so that we can compare fairly across them. No high power channels will dominate motifs!

def sliding_zscore(arr, window_size):
    mean = uniform_filter1d(arr, size=window_size, mode='nearest')
    sq_mean = uniform_filter1d(arr**2, size=window_size, mode='nearest')
    std = np.sqrt(sq_mean - mean**2)
    z = (arr - mean) / std
    return z


def legacy_sliding_zscore(power_cube, bin_width=1000, target_band=None):
    """
    Applies a sliding z-score normalization to a 3D power cube (channels x bands x time).
    Only normalizes a specific frequency band if target_band is specified (int).
    """
    normalized_power_cube = np.zeros_like(power_cube)
    half_bin = bin_width // 2
    n_channels, n_bands, n_times = power_cube.shape

    for ch in range(n_channels):
        print(f"Working on Channel {ch}")
        for band in range(n_bands):
            if (target_band is not None) and (band != target_band):
                continue
            for i in range(n_times):
                start = max(0, i - half_bin)
                end = min(n_times, i + half_bin)
                window = power_cube[ch, band, start:end]

                data_mean = np.mean(window)
                data_std = np.std(window)
                if data_std == 0:
                    print(f"  ⚠️ Std=0 at ch={ch}, band={band}, time={i}")
                    data_std = 1e-6  # Prevent divide-by-zero

                normalized_power_cube[ch, band, i] = (power_cube[ch, band, i] - data_mean) / data_std

    return normalized_power_cube


normalized_power_cube = np.zeros_like(power_cube)
bin_width = 1000  # Must be odd to center the window (if not, close enough is fine)

for ch in range(power_cube.shape[0]): #power_cube.shape[0]cha
    print(f"Channel {ch}")
    for band in range(power_cube.shape[1]):
        normalized_power_cube[ch, band, :] = sliding_zscore(power_cube[ch, band, :], bin_width)

#Legacy, works but is extremely slow computationally.

#print(np.allclose(test_normPC[0, 0, 10000:20000], normalized_power_cube[0, 0, 10000:20000], atol=1e-5))  # Should be True


Channel 0
Channel 1
Channel 2
Channel 3
Channel 4
Channel 5
Channel 6
Channel 7
Channel 8
Channel 9
Channel 10
Channel 11
Channel 12
Channel 13
Channel 14
Channel 15
Channel 16
Channel 17
Channel 18
Channel 19
Channel 20
Channel 21
Channel 22
Channel 23
Channel 24
Channel 25
Channel 26
Channel 27
Channel 28
Channel 29
Channel 30
Channel 31
Channel 32
Channel 33
Channel 34
Channel 35
Channel 36
Channel 37
Channel 38
Channel 39
Channel 40
Channel 41
Channel 42
Channel 43
Channel 44
Channel 45
Channel 46
Channel 47
Channel 48
Channel 49
Channel 50
Channel 51
Channel 52
Channel 53
Channel 54
Channel 55
Channel 56
Channel 57
Channel 58
Channel 59
Channel 60
Channel 61
Channel 62
Channel 63
Channel 64
Channel 65
Channel 66
Channel 67
Channel 68
Channel 69
Channel 70
Channel 71
Channel 72
Channel 73
Channel 74
Channel 75
Channel 76
Channel 77
Channel 78
Channel 79
Channel 80
Channel 81
Channel 82
Channel 83
Channel 84
Channel 85
Channel 86
Channel 87
Channel 88
Channel 89
Channel 90
Channel 9

In [7]:

#Now we work to make all values positive:
for ch in range(normalized_power_cube.shape[0]):
    print(ch)
    for band in range(normalized_power_cube.shape[1]):
        min_val = np.min(normalized_power_cube[ch, band, :])
        normalized_power_cube[ch, band, :] -= min_val



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118


In [25]:
start_time = time.time()


K = 28
L = 975
Lambda = 0.0005

#[W, H, cost, loadings, power] = seqnmf(example_data, K, L, Lambda)
[W, H, cost, loadings, power] = seqnmf(normalized_power_cube[:, 2, 0:6660], K, L=1000, Lambda=Lambda)

end_time = time.time()
runtime_seconds = end_time - start_time
print("Runtime was %s seconds" %runtime_seconds)

KeyboardInterrupt: 

In [20]:
plt.plot(power_cube[0, 2, 0:100000])

In [22]:
power_cube.shape

(119, 6, 3600000)

In [17]:
example_data.shape

(75, 666)